In [135]:
import os
import librosa
import numpy as np
from joblib import Parallel, delayed
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras


In [136]:
def extract_features(file_path):
    try:
        y, sr = librosa.load(file_path, duration=30)
        chroma_stft_mean = np.mean(librosa.feature.chroma_stft(y=y, sr=sr))
        mfccs_mean = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13))
        spectral_centroid_mean = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr))
        spectral_rolloff_mean = np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr))
        spectral_contrast_mean = np.mean(librosa.feature.spectral_contrast(y=y, sr=sr))
        return chroma_stft_mean, mfccs_mean, spectral_centroid_mean, spectral_rolloff_mean, spectral_contrast_mean
    except Exception as e:
        print(f"Error processing file: {file_path}. Skipping.")
        return None

In [137]:
def extract_features_parallel(file_paths):
    X = []
    processed_files = []
    for file_path in file_paths:
        features = extract_features(file_path)
        if features is not None:
            X.append(features)
            processed_files.append(file_path)
    return np.array(X), processed_files


In [138]:
# Main code
data_directory = r"D:\Machine Learning Project\DATA_SETS\Data\genres"
audio_files = []
# Load music files from dataset
for genre_label in os.listdir(data_directory):
    genre_path = os.path.join(data_directory, genre_label)
    audio_files.extend([os.path.join(genre_path, file_name) for file_name in os.listdir(genre_path) if file_name.endswith('.wav')])


In [139]:
X, processed_files = extract_features_parallel(audio_files)
# Load genre labels
y = [os.path.basename(os.path.dirname(file_path)) for file_path in processed_files]


In [140]:
 #Encode genre labels to numerical values
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)


In [141]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)


In [142]:
# Create and train a neural network model
model = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(len(label_encoder.classes_), activation='softmax')
])


In [143]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [207]:
# Train the model with multiple epochs
epochs = 50
for epoch in range(epochs):
    model.fit(X_train, y_train, epochs=1, batch_size=32, validation_split=0.2, verbose=0)
    accuracy = model.evaluate(X_test, y_test, verbose=0)[1]
    print(f"Epoch {epoch + 1}, Accuracy: {accuracy}")


Epoch 1, Accuracy: 0.3222222328186035
Epoch 2, Accuracy: 0.35555556416511536
Epoch 3, Accuracy: 0.3222222328186035
Epoch 4, Accuracy: 0.3499999940395355
Epoch 5, Accuracy: 0.3611111044883728
Epoch 6, Accuracy: 0.3333333432674408
Epoch 7, Accuracy: 0.31111112236976624
Epoch 8, Accuracy: 0.35555556416511536
Epoch 9, Accuracy: 0.3333333432674408
Epoch 10, Accuracy: 0.3222222328186035
Epoch 11, Accuracy: 0.3222222328186035
Epoch 12, Accuracy: 0.33888888359069824
Epoch 13, Accuracy: 0.33888888359069824
Epoch 14, Accuracy: 0.35555556416511536
Epoch 15, Accuracy: 0.3333333432674408
Epoch 16, Accuracy: 0.35555556416511536
Epoch 17, Accuracy: 0.33888888359069824
Epoch 18, Accuracy: 0.35555556416511536
Epoch 19, Accuracy: 0.3222222328186035
Epoch 20, Accuracy: 0.3499999940395355
Epoch 21, Accuracy: 0.3055555522441864
Epoch 22, Accuracy: 0.3611111044883728
Epoch 23, Accuracy: 0.33888888359069824
Epoch 24, Accuracy: 0.33888888359069824
Epoch 25, Accuracy: 0.3722222149372101
Epoch 26, Accuracy: 0.3

In [208]:
# Save the trained model
model.save("music_genre_model.h5")


C:\Users\saura\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [209]:
# Load the model for prediction on new audio files
loaded_model = keras.models.load_model("music_genre_model.h5")


In [210]:
# Test the model on new audio files
new_audio_files = [r"C:\Users\saura\Downloads\Tum Hi Ho.wav"]


In [211]:
X_new, processed_files = extract_features_parallel(new_audio_files)


In [212]:
# Make predictions on new audio files
y_new_pred = loaded_model.predict(X_new)
y_new_pred_genre = label_encoder.inverse_transform(np.argmax(y_new_pred, axis=-1))


1/1 [==============================] - 0s 54ms/step


In [213]:
# Print the predicted genre for each new audio file
for file_path, genre in zip(new_audio_files, y_new_pred_genre):
    print(f"File: {file_path} --> Predicted Genre: {genre}")


File: C:\Users\saura\Downloads\Tum Hi Ho.wav --> Predicted Genre: jazz
